# TRL Library Tutorial: Your Complete Toolkit for LLM Alignment

TRL is like a Swiss Army knife for RLHF - everything you need in one library!

## What You'll Learn

By the end of this notebook, you'll understand:
- The toolbox analogy: what TRL provides and why
- SFTTrainer: Supervised fine-tuning made easy
- RewardTrainer: Training reward models
- PPOTrainer: Full RLHF training
- DPOTrainer: The simpler alternative
- Practical tips for each trainer

**Prerequisites:** Notebooks 1-4 (RLHF basics through DPO)

**Time:** ~35 minutes

---
## The Big Picture: The Toolbox Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE TRL TOOLBOX ANALOGY                               │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine building a house...                                  │
    │                                                                │
    │  WITHOUT TRL (manual construction):                           │
    │    - Cut each board by hand                                   │
    │    - Make your own nails                                      │
    │    - Design each joint from scratch                           │
    │    → Takes months, easy to make mistakes!                     │
    │                                                                │
    │  WITH TRL (professional toolkit):                             │
    │    🔧 SFTTrainer = Foundation (instruction tuning)            │
    │    🔧 RewardTrainer = Measuring stick (what's good?)          │
    │    🔧 PPOTrainer = Power tools (optimize with RL)             │
    │    🔧 DPOTrainer = Quick assembly (direct training)           │
    │    → Professional results in days!                            │
    │                                                                │
    │  TRL PHILOSOPHY:                                              │
    │    "Don't reinvent the wheel - use proven tools!"            │
    │    "Same patterns that trained ChatGPT, Claude, etc."        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, FancyArrowPatch
import warnings
warnings.filterwarnings('ignore')

# Check TRL availability
TRL_AVAILABLE = False
try:
    import trl
    from trl import SFTTrainer, SFTConfig, DPOTrainer, DPOConfig
    TRL_AVAILABLE = True
    print(f"✓ TRL version {trl.__version__} is installed!")
except ImportError:
    print("✗ TRL not installed.")
    print("  Install with: pip install trl transformers peft accelerate")

# Check transformers
try:
    import transformers
    print(f"✓ Transformers version {transformers.__version__}")
except ImportError:
    print("✗ Transformers not installed.")

# Check PEFT
try:
    import peft
    print(f"✓ PEFT version {peft.__version__}")
except ImportError:
    print("✗ PEFT not installed.")

In [ ]:
# Visualize TRL components

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('TRL Library: Your Complete Alignment Toolkit', fontsize=16, fontweight='bold')

# Main TRL box
trl_box = FancyBboxPatch((1, 1), 12, 9.5, boxstyle="round,pad=0.1",
                          facecolor='#f5f5f5', edgecolor='#333', linewidth=3)
ax.add_patch(trl_box)
ax.text(7, 10, 'TRL (Transformer Reinforcement Learning)', ha='center', 
        fontsize=14, fontweight='bold')

# Component boxes
components = [
    (1.5, 7, 'SFTTrainer', 'Supervised\nFine-Tuning', '#c8e6c9', '#388e3c',
     'Foundation'),
    (6, 7, 'RewardTrainer', 'Reward Model\nTraining', '#fff3e0', '#f57c00',
     'Scoring'),
    (10.5, 7, 'PPOTrainer', 'PPO for\nLanguage Models', '#e1bee7', '#7b1fa2',
     'RLHF'),
    (3.75, 3, 'DPOTrainer', 'Direct Preference\nOptimization', '#bbdefb', '#1976d2',
     'Simpler'),
    (8.25, 3, 'ORPOTrainer', 'Odds Ratio\nPreference', '#ffcdd2', '#d32f2f',
     'Combined'),
]

for x, y, name, desc, fcolor, ecolor, label in components:
    box = FancyBboxPatch((x, y), 3, 2.5, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax.add_patch(box)
    ax.text(x + 1.5, y + 2, name, ha='center', fontsize=10, fontweight='bold', color=ecolor)
    ax.text(x + 1.5, y + 1, desc, ha='center', fontsize=9)
    ax.text(x + 1.5, y + 0.3, f'({label})', ha='center', fontsize=8, style='italic', color='#666')

# Flow arrows showing typical pipeline
ax.annotate('', xy=(5.9, 8.25), xytext=(4.6, 8.25),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(10.4, 8.25), xytext=(9.1, 8.25),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# OR arrow to DPO
ax.annotate('', xy=(5.25, 5.6), xytext=(3, 6.9),
            arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2', linestyle='--'))
ax.text(3.5, 6.5, 'OR', fontsize=10, color='#1976d2', fontweight='bold')

# Pipeline labels
ax.text(7.5, 9.2, 'Traditional RLHF Pipeline →', ha='center', fontsize=10, color='#666')
ax.text(5.25, 6.3, 'Direct Path (skip RM!)', ha='center', fontsize=9, color='#1976d2')

plt.tight_layout()
plt.show()

print("\nTRL COMPONENT SUMMARY:")
print("  • SFTTrainer: Turn base model into instruction-following model")
print("  • RewardTrainer: Train a model to score response quality")
print("  • PPOTrainer: Optimize with reinforcement learning")
print("  • DPOTrainer: Direct training on preferences (simpler!)")
print("  • ORPOTrainer: Combined SFT + preference in one step")

---
## TRL Data Formats: Understanding the Input

```
    ┌────────────────────────────────────────────────────────────────┐
    │              TRL DATA FORMATS                                  │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  SFTTrainer (instruction data):                               │
    │    {                                                          │
    │      "text": "User: How do I...\nAssistant: Here's how..."   │
    │    }                                                          │
    │    OR with chat template:                                     │
    │    {                                                          │
    │      "messages": [                                           │
    │        {"role": "user", "content": "How do I..."},          │
    │        {"role": "assistant", "content": "Here's how..."}    │
    │      ]                                                        │
    │    }                                                          │
    │                                                                │
    │  RewardTrainer (preference pairs):                            │
    │    {                                                          │
    │      "chosen": "Good response text",                        │
    │      "rejected": "Bad response text"                        │
    │    }                                                          │
    │                                                                │
    │  DPOTrainer (preference pairs with prompt):                   │
    │    {                                                          │
    │      "prompt": "User question",                             │
    │      "chosen": "Preferred response",                        │
    │      "rejected": "Non-preferred response"                   │
    │    }                                                          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Demonstrate data formats

print("TRL DATA FORMAT EXAMPLES")
print("="*60)

# SFT format (text)
sft_text_example = {
    "text": """### Human: What is machine learning?

### Assistant: Machine learning is a subset of artificial intelligence where computers learn patterns from data rather than being explicitly programmed. Instead of writing rules, we show the algorithm examples and it learns to make predictions or decisions."""
}

print("\n1. SFT FORMAT (text-based):")
print("-"*40)
print(sft_text_example["text"][:200] + "...")

# SFT format (messages)
sft_messages_example = {
    "messages": [
        {"role": "user", "content": "What is machine learning?"},
        {"role": "assistant", "content": "Machine learning is a subset of AI..."}
    ]
}

print("\n2. SFT FORMAT (messages):")
print("-"*40)
for msg in sft_messages_example["messages"]:
    print(f"  {msg['role']}: {msg['content'][:50]}...")

# Preference format
preference_example = {
    "prompt": "Explain quantum computing in simple terms.",
    "chosen": "Quantum computing uses quantum bits (qubits) that can be 0, 1, or both at once. This lets quantum computers solve certain problems much faster than regular computers.",
    "rejected": "Quantum computing is very complicated and uses advanced physics. It's probably too technical for you to understand."
}

print("\n3. PREFERENCE FORMAT (DPO/Reward):")
print("-"*40)
print(f"  Prompt: {preference_example['prompt']}")
print(f"  Chosen: {preference_example['chosen'][:60]}...")
print(f"  Rejected: {preference_example['rejected'][:60]}...")

print("\n" + "="*60)

---
## Component 1: SFTTrainer - Building the Foundation

```
    ┌────────────────────────────────────────────────────────────────┐
    │              SFTTrainer: SUPERVISED FINE-TUNING                │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PURPOSE:                                                     │
    │    Transform base LLM → Instruction-following model           │
    │                                                                │
    │  ANALOGY: Teaching someone a new job                          │
    │    Base model = Smart person who reads a lot                 │
    │    SFT = Showing them examples of good work                  │
    │    Result = Employee who knows how to do the job             │
    │                                                                │
    │  KEY FEATURES:                                                │
    │    • Automatic dataset formatting                            │
    │    • LoRA/QLoRA support (efficient training)                 │
    │    • Multi-GPU support                                       │
    │    • Packing for efficient training                          │
    │                                                                │
    │  WHEN TO USE:                                                 │
    │    • First step in any LLM training pipeline                 │
    │    • Converting base model to chat model                     │
    │    • Domain adaptation                                       │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# SFTTrainer example code

print("SFTTrainer COMPLETE EXAMPLE")
print("="*60)

sft_code = '''
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from datasets import load_dataset

# 1. Load base model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"  # Or any base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load instruction dataset
# Format: {"text": "Human: ...\\nAssistant: ..."}
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

# 3. Configure LoRA for efficient training
peft_config = LoraConfig(
    r=16,                         # Rank of update matrices
    lora_alpha=32,                # Scaling factor
    lora_dropout=0.05,            # Dropout for regularization
    target_modules=[              # Which layers to adapt
        "q_proj", "v_proj",       # Attention projections
        "k_proj", "o_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# 4. Configure SFT training
training_args = SFTConfig(
    output_dir="./sft_output",
    
    # Training parameters
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    # Sequence settings
    max_seq_length=512,
    packing=True,                  # Pack short examples together
    
    # Optimization
    learning_rate=2e-4,
    warmup_steps=100,
    
    # Logging
    logging_steps=10,
    save_steps=500,
)

# 5. Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

# 6. Train!
trainer.train()

# 7. Save the model
trainer.save_model("./sft_final")
'''

print(sft_code)
print("="*60)

In [ ]:
# Visualize SFT process

fig, ax = plt.subplots(figsize=(14, 6))
ax.set_xlim(0, 14)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('SFTTrainer: From Base Model to Instruction Model', fontsize=14, fontweight='bold')

# Base model
base_box = FancyBboxPatch((0.5, 2.5), 3, 3, boxstyle="round,pad=0.1",
                           facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax.add_patch(base_box)
ax.text(2, 5, 'Base LLM', ha='center', fontsize=11, fontweight='bold', color='#1976d2')
ax.text(2, 4, '(GPT-2, Llama, etc.)', ha='center', fontsize=9)
ax.text(2, 3.3, 'Knows language,', ha='center', fontsize=9, color='#666')
ax.text(2, 2.8, 'but not instructions', ha='center', fontsize=9, color='#666')

# Instruction data
data_box = FancyBboxPatch((5, 5.5), 4, 2, boxstyle="round,pad=0.1",
                           facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(data_box)
ax.text(7, 6.8, 'Instruction Data', ha='center', fontsize=10, fontweight='bold', color='#f57c00')
ax.text(7, 6, '(Question, Answer pairs)', ha='center', fontsize=9)

# SFTTrainer
sft_box = FancyBboxPatch((5, 2.5), 4, 2.5, boxstyle="round,pad=0.1",
                          facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(sft_box)
ax.text(7, 4.5, 'SFTTrainer', ha='center', fontsize=12, fontweight='bold', color='#388e3c')
ax.text(7, 3.7, '+ LoRA', ha='center', fontsize=10)
ax.text(7, 3, 'Supervised Learning', ha='center', fontsize=9, color='#666')

# Aligned model
aligned_box = FancyBboxPatch((10.5, 2.5), 3, 3, boxstyle="round,pad=0.1",
                              facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=2)
ax.add_patch(aligned_box)
ax.text(12, 5, 'Instruction', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
ax.text(12, 4.5, 'Model', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
ax.text(12, 3.7, 'Follows instructions!', ha='center', fontsize=9, color='#666')
ax.text(12, 3.2, 'Ready for RLHF', ha='center', fontsize=9, color='#666')

# Arrows
ax.annotate('', xy=(4.9, 4), xytext=(3.6, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(7, 5.4), xytext=(7, 5.1),
            arrowprops=dict(arrowstyle='->', lw=2, color='#f57c00'))
ax.annotate('', xy=(10.4, 4), xytext=(9.1, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))

# Key points
ax.text(7, 0.8, 'Key: SFT teaches the model HOW to respond to instructions', 
        ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

---
## Component 2: RewardTrainer - Training the Judge

```
    ┌────────────────────────────────────────────────────────────────┐
    │              RewardTrainer: REWARD MODEL TRAINING              │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PURPOSE:                                                     │
    │    Train a model to score response quality                   │
    │                                                                │
    │  ANALOGY: Training a judge for a contest                      │
    │    Show pairs: "This response is better than that one"       │
    │    Judge learns: What makes a response good?                 │
    │    Result: Can score ANY new response                        │
    │                                                                │
    │  KEY FEATURES:                                                │
    │    • Bradley-Terry loss (pairwise comparisons)               │
    │    • Works with any transformer model                        │
    │    • Outputs scalar reward per (prompt, response)            │
    │                                                                │
    │  WHEN TO USE:                                                 │
    │    • RLHF with PPO (need reward model)                       │
    │    • When you have preference data                           │
    │    • NOT needed for DPO!                                     │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# RewardTrainer example code

print("RewardTrainer COMPLETE EXAMPLE")
print("="*60)

reward_code = '''
from trl import RewardTrainer, RewardConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

# 1. Load model FOR SEQUENCE CLASSIFICATION (not causal LM!)
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,              # Single scalar output!
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load preference dataset
# Format: {"chosen": "good response", "rejected": "bad response"}
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:10000]")

# 3. Configure reward training
training_args = RewardConfig(
    output_dir="./reward_model",
    
    # Training settings
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    # Sequence settings
    max_length=512,
    
    # Optimization
    learning_rate=1e-5,
    
    # Logging
    logging_steps=10,
)

# 4. Create trainer
trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# 5. Train!
trainer.train()

# 6. Save
trainer.save_model("./reward_model_final")

# 7. Use the reward model
def get_reward(prompt, response):
    text = f"{prompt}\\n{response}"
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.item()  # Scalar reward
'''

print(reward_code)
print("="*60)

---
## Component 3: PPOTrainer - Full RLHF

```
    ┌────────────────────────────────────────────────────────────────┐
    │              PPOTrainer: REINFORCEMENT LEARNING                │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PURPOSE:                                                     │
    │    Optimize language model using reward model feedback       │
    │                                                                │
    │  THE LOOP:                                                    │
    │    1. Generate responses with current policy                 │
    │    2. Score responses with reward model                      │
    │    3. Update policy with PPO to maximize reward              │
    │    4. KL penalty keeps model from going crazy                │
    │    5. Repeat!                                                │
    │                                                                │
    │  KEY FEATURES:                                                │
    │    • Automatic value head management                         │
    │    • KL penalty computation                                  │
    │    • Advantage estimation                                    │
    │    • Response generation                                     │
    │                                                                │
    │  WHEN TO USE:                                                 │
    │    • Online learning (generate → feedback → improve)         │
    │    • When you need flexible reward shaping                   │
    │    • When you have a trained reward model                    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# PPOTrainer example code

print("PPOTrainer COMPLETE EXAMPLE")
print("="*60)

ppo_code = '''
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer
from tqdm import tqdm

# 1. Load model WITH VALUE HEAD (TRL special wrapper)
model = AutoModelForCausalLMWithValueHead.from_pretrained("./sft_model")
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("./sft_model")

tokenizer = AutoTokenizer.from_pretrained("./sft_model")
tokenizer.pad_token = tokenizer.eos_token

# 2. Load pre-trained reward model
reward_model = AutoModelForSequenceClassification.from_pretrained("./reward_model")

# 3. Configure PPO
ppo_config = PPOConfig(
    # Core PPO settings
    learning_rate=1.41e-5,
    batch_size=64,
    mini_batch_size=4,
    ppo_epochs=4,
    
    # KL penalty settings
    kl_penalty="kl",
    init_kl_coef=0.2,     # β coefficient
    target_kl=6.0,        # Target KL divergence
    
    # Generation settings
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
)

# 4. Create PPO trainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,     # For KL penalty
    tokenizer=tokenizer,
)

# 5. Training loop
for epoch in range(3):
    for batch in tqdm(dataloader):
        # Get prompts
        query_tensors = [tokenizer.encode(q, return_tensors="pt").squeeze() 
                        for q in batch["prompt"]]
        
        # Generate responses with current policy
        response_tensors = ppo_trainer.generate(
            query_tensors,
            return_prompt=False,
        )
        
        # Decode responses
        responses = [tokenizer.decode(r, skip_special_tokens=True) 
                    for r in response_tensors]
        
        # Get rewards from reward model
        rewards = []
        for prompt, response in zip(batch["prompt"], responses):
            text = f"{prompt}\\n{response}"
            inputs = tokenizer(text, return_tensors="pt", truncation=True)
            with torch.no_grad():
                score = reward_model(**inputs).logits.item()
            rewards.append(torch.tensor(score))
        
        # PPO step - this does the magic!
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        
        # Log
        print(f"Mean reward: {stats[\'ppo/mean_scores\']:.3f}")
        print(f"KL divergence: {stats[\'objective/kl\']:.3f}")

# 6. Save final model
model.save_pretrained("./ppo_aligned_model")
'''

print(ppo_code)
print("="*60)

In [ ]:
# Visualize PPO training loop

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('PPOTrainer: The RLHF Training Loop', fontsize=16, fontweight='bold')

# Central loop
center_x, center_y = 7, 5

# Steps around the loop
steps = [
    (2, 7, 'Generate', 'Create responses\nwith current policy', '#c8e6c9', '#388e3c'),
    (9, 7, 'Score', 'Reward model\nrates responses', '#e1bee7', '#7b1fa2'),
    (9, 2.5, 'Update', 'PPO optimizes\npolicy weights', '#fff3e0', '#f57c00'),
    (2, 2.5, 'Constrain', 'KL penalty\nkeeps stability', '#ffcdd2', '#d32f2f'),
]

for x, y, title, desc, fcolor, ecolor in steps:
    box = FancyBboxPatch((x, y), 3, 2, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax.add_patch(box)
    ax.text(x + 1.5, y + 1.5, title, ha='center', fontsize=11, fontweight='bold', color=ecolor)
    ax.text(x + 1.5, y + 0.6, desc, ha='center', fontsize=9)

# Arrows forming the loop
ax.annotate('', xy=(8.9, 8), xytext=(5.1, 8),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(10.5, 7), xytext=(10.5, 4.6),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(5.1, 3.5), xytext=(8.9, 3.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(3.5, 4.6), xytext=(3.5, 7),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# Central "REPEAT" text
circle = Circle((7, 5), 1, facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax.add_patch(circle)
ax.text(7, 5.2, 'REPEAT', ha='center', fontsize=10, fontweight='bold', color='#1976d2')
ax.text(7, 4.7, '1000s of', ha='center', fontsize=8)
ax.text(7, 4.3, 'iterations', ha='center', fontsize=8)

# Models involved
ax.text(3.5, 0.8, 'Models: Policy (being trained) + Reference (frozen) + Reward Model', 
        ha='center', fontsize=10, color='#666')

plt.tight_layout()
plt.show()

---
## Component 4: DPOTrainer - The Simpler Path

```
    ┌────────────────────────────────────────────────────────────────┐
    │              DPOTrainer: DIRECT PREFERENCE OPTIMIZATION        │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PURPOSE:                                                     │
    │    Train on preferences WITHOUT a reward model!              │
    │                                                                │
    │  WHY DPO?                                                     │
    │    • No reward model training needed                         │
    │    • No complex PPO loop                                     │
    │    • Just supervised learning with special loss              │
    │    • Often works just as well as PPO!                        │
    │                                                                │
    │  THE LOSS:                                                    │
    │    L = -log σ(β × (log π(y_w)/π_ref(y_w)                     │
    │                  - log π(y_l)/π_ref(y_l)))                   │
    │                                                                │
    │    "Increase prob of chosen, decrease prob of rejected,      │
    │     but don't drift too far from reference"                  │
    │                                                                │
    │  WHEN TO USE:                                                 │
    │    • You have preference pairs                               │
    │    • You want simplicity                                     │
    │    • No need for online learning                             │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# DPOTrainer example code

print("DPOTrainer COMPLETE EXAMPLE")
print("="*60)

dpo_code = '''
from trl import DPOTrainer, DPOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from datasets import load_dataset

# 1. Load SFT model (starting point)
model = AutoModelForCausalLM.from_pretrained(
    "./sft_model",
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained("./sft_model")
tokenizer.pad_token = tokenizer.eos_token

# 2. Load preference dataset
# Format: {"prompt": "...", "chosen": "...", "rejected": "..."}
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:5000]")

# 3. Optional: LoRA for efficient training
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# 4. Configure DPO training
training_args = DPOConfig(
    output_dir="./dpo_output",
    
    # DPO-specific settings
    beta=0.1,                      # KL penalty coefficient
    loss_type="sigmoid",           # Standard DPO loss
    
    # Training settings
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    # Sequence settings  
    max_length=512,
    max_prompt_length=256,
    
    # Optimization
    learning_rate=5e-5,
    warmup_ratio=0.1,
    
    # Logging
    logging_steps=10,
    save_steps=500,
)

# 5. Create DPO trainer
# Reference model is created automatically from initial weights!
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,  # Optional: use LoRA
)

# 6. Train!
trainer.train()

# 7. Save
trainer.save_model("./dpo_aligned_model")
'''

print(dpo_code)
print("="*60)

In [ ]:
# Visualize PPO vs DPO comparison

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Left: PPO pipeline
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 12)
ax1.axis('off')
ax1.set_title('PPOTrainer Pipeline\n(3 stages, complex)', fontsize=12, fontweight='bold', color='#f57c00')

ppo_stages = [
    (3.5, 9.5, 'SFT', 'SFTTrainer', '#c8e6c9', '#388e3c'),
    (3.5, 6.5, 'Reward\nModel', 'RewardTrainer', '#e1bee7', '#7b1fa2'),
    (3.5, 3.5, 'PPO', 'PPOTrainer', '#fff3e0', '#f57c00'),
    (3.5, 0.5, 'Aligned\nModel', '✓', '#bbdefb', '#1976d2'),
]

for x, y, title, subtitle, fcolor, ecolor in ppo_stages:
    box = FancyBboxPatch((x, y), 3, 2, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax1.add_patch(box)
    ax1.text(x + 1.5, y + 1.3, title, ha='center', fontsize=10, fontweight='bold')
    ax1.text(x + 1.5, y + 0.5, subtitle, ha='center', fontsize=9, color='#666')

# Arrows
for y in [9.4, 6.4, 3.4]:
    ax1.annotate('', xy=(5, y-0.5), xytext=(5, y),
                 arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# Complexity indicators
ax1.text(8, 10.3, '😊', fontsize=16)
ax1.text(8, 7.3, '😓', fontsize=16)
ax1.text(8, 4.3, '😵', fontsize=16)

# Right: DPO pipeline
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 12)
ax2.axis('off')
ax2.set_title('DPOTrainer Pipeline\n(2 stages, simple!)', fontsize=12, fontweight='bold', color='#388e3c')

dpo_stages = [
    (3.5, 8, 'SFT', 'SFTTrainer', '#c8e6c9', '#388e3c'),
    (3.5, 4, 'DPO', 'DPOTrainer', '#bbdefb', '#1976d2'),
    (3.5, 0.5, 'Aligned\nModel', '✓', '#c8e6c9', '#388e3c'),
]

for x, y, title, subtitle, fcolor, ecolor in dpo_stages:
    box = FancyBboxPatch((x, y), 3, 2.5 if 'DPO' in title or 'SFT' in title else 2, 
                          boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax2.add_patch(box)
    ax2.text(x + 1.5, y + 1.5 if 'DPO' in title or 'SFT' in title else y + 1.3, 
             title, ha='center', fontsize=10, fontweight='bold')
    ax2.text(x + 1.5, y + 0.7 if 'DPO' in title or 'SFT' in title else y + 0.5, 
             subtitle, ha='center', fontsize=9, color='#666')

# Arrows
ax2.annotate('', xy=(5, 6.9), xytext=(5, 7.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax2.annotate('', xy=(5, 3), xytext=(5, 3.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# Happy faces
ax2.text(8, 9, '😊', fontsize=16)
ax2.text(8, 5, '😎', fontsize=20)

# Skip arrow
ax2.text(8.5, 7, 'Skip\nReward\nModel!', fontsize=9, ha='center', color='#d32f2f', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nDPO ADVANTAGES:")
print("  • No reward model training (saves time!)")
print("  • No complex PPO loop (easier to debug)")
print("  • Often matches PPO performance")
print("  • Recommended starting point for most projects!")

---
## Practical Tips for Each Trainer

```
    ┌────────────────────────────────────────────────────────────────┐
    │              PRACTICAL TIPS                                    │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  SFTTrainer Tips:                                             │
    │    • Use packing=True for efficiency                         │
    │    • Start with small max_seq_length (256-512)               │
    │    • Use LoRA unless you have lots of GPUs                   │
    │    • Quality of data >> quantity of data                     │
    │                                                                │
    │  RewardTrainer Tips:                                          │
    │    • Use same base model as your SFT model                   │
    │    • 5K-10K preference pairs is often enough                 │
    │    • Watch for length bias in rewards                        │
    │                                                                │
    │  PPOTrainer Tips:                                             │
    │    • Start with default hyperparameters                      │
    │    • Monitor KL divergence (should stay < 10)                │
    │    • Use small batch sizes to start                          │
    │    • Train for fewer steps than you think                    │
    │                                                                │
    │  DPOTrainer Tips:                                             │
    │    • beta=0.1 is a good default                              │
    │    • Use LoRA for memory efficiency                          │
    │    • Reference model created automatically                    │
    │    • Often better to start here instead of PPO               │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Quick reference: hyperparameters

print("HYPERPARAMETER QUICK REFERENCE")
print("="*60)

hyperparams = {
    'SFTTrainer': {
        'learning_rate': '2e-4 (with LoRA) / 2e-5 (full)',
        'max_seq_length': '512-1024',
        'batch_size': '4-8 per GPU',
        'epochs': '1-3',
        'packing': 'True (recommended)',
    },
    'RewardTrainer': {
        'learning_rate': '1e-5',
        'max_length': '512',
        'batch_size': '4 per GPU',
        'epochs': '1',
    },
    'PPOTrainer': {
        'learning_rate': '1.41e-5',
        'batch_size': '64-128',
        'mini_batch_size': '4-8',
        'ppo_epochs': '4',
        'init_kl_coef': '0.2',
        'target_kl': '6.0',
    },
    'DPOTrainer': {
        'learning_rate': '5e-5 (with LoRA) / 5e-7 (full)',
        'beta': '0.1',
        'max_length': '512',
        'max_prompt_length': '256',
        'epochs': '1-3',
    },
}

for trainer, params in hyperparams.items():
    print(f"\n{trainer}:")
    for param, value in params.items():
        print(f"  {param}: {value}")

print("\n" + "="*60)

---
## Common Data Sources

```
    ┌────────────────────────────────────────────────────────────────┐
    │              DATASETS FOR EACH STAGE                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  SFT (Instruction Data):                                      │
    │    • timdettmers/openassistant-guanaco                       │
    │    • databricks/databricks-dolly-15k                         │
    │    • HuggingFaceH4/ultrachat_200k                            │
    │    • tatsu-lab/alpaca                                        │
    │                                                                │
    │  Preference Data (DPO/Reward):                                │
    │    • Anthropic/hh-rlhf (human preferences)                   │
    │    • HuggingFaceH4/ultrafeedback_binarized                   │
    │    • argilla/ultrafeedback-binarized-preferences             │
    │    • stanfordnlp/SHP (Stack Exchange preferences)            │
    │                                                                │
    │  TIP: Start small (1K-5K examples) to validate pipeline!     │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Show example of loading datasets

print("LOADING DATASETS FOR TRL")
print("="*60)

loading_code = '''
from datasets import load_dataset

# For SFT: Instruction-following data
sft_dataset = load_dataset(
    "timdettmers/openassistant-guanaco", 
    split="train"
)
print(f"SFT dataset: {len(sft_dataset)} examples")
print(f"Columns: {sft_dataset.column_names}")
# Column: "text" with Human/Assistant conversation

# For DPO/Reward: Preference data
preference_dataset = load_dataset(
    "Anthropic/hh-rlhf",
    split="train[:5000]"  # Start small!
)
print(f"\\nPreference dataset: {len(preference_dataset)} examples")
print(f"Columns: {preference_dataset.column_names}")
# Columns: "chosen", "rejected"

# Convert to DPO format if needed (add "prompt" column)
def extract_prompt(example):
    """Extract prompt from chosen response."""
    # hh-rlhf has full conversation in chosen/rejected
    # Extract just the human turn as prompt
    chosen = example["chosen"]
    # Find the assistant response start
    if "\\nAssistant:" in chosen:
        parts = chosen.split("\\nAssistant:")
        prompt = parts[0].replace("Human: ", "")
        chosen_response = parts[1]
    else:
        prompt = chosen[:100]
        chosen_response = chosen
    return {
        "prompt": prompt,
        "chosen": chosen_response,
        "rejected": example["rejected"].split("\\nAssistant:")[-1] if "\\nAssistant:" in example["rejected"] else example["rejected"]
    }

dpo_dataset = preference_dataset.map(extract_prompt)
'''

print(loading_code)
print("="*60)

---
## Summary: TRL Quick Reference

### Which Trainer to Use?

| Trainer | Use When | Data Format |
|---------|----------|-------------|
| **SFTTrainer** | Starting with base model | `{"text": "..."}` or `{"messages": [...]}` |
| **RewardTrainer** | Need explicit reward model | `{"chosen": "...", "rejected": "..."}` |
| **PPOTrainer** | Online RLHF training | Prompts + Reward Model |
| **DPOTrainer** | Simple preference learning | `{"prompt": "...", "chosen": "...", "rejected": "..."}` |

### Recommended Pipeline

```
For most projects:  SFTTrainer → DPOTrainer → Done! ✓

For advanced use:   SFTTrainer → RewardTrainer → PPOTrainer
```

### Key Imports

```python
# Core trainers
from trl import SFTTrainer, SFTConfig
from trl import DPOTrainer, DPOConfig
from trl import PPOTrainer, PPOConfig
from trl import RewardTrainer, RewardConfig

# For PPO
from trl import AutoModelForCausalLMWithValueHead

# Supporting libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from datasets import load_dataset
```

---
## Test Your Understanding

**1. What is the main difference between PPOTrainer and DPOTrainer?**
<details>
<summary>Click to reveal answer</summary>
PPOTrainer requires a separate reward model and performs online RL training (generate → score → update). DPOTrainer skips the reward model entirely and directly trains on preference pairs using a special loss function. DPO is simpler and often just as effective!
</details>

**2. What data format does DPOTrainer expect?**
<details>
<summary>Click to reveal answer</summary>
DPOTrainer expects preference pairs with three fields:
- "prompt": The user question/input
- "chosen": The preferred/better response
- "rejected": The non-preferred/worse response
</details>

**3. Why is LoRA important for TRL training?**
<details>
<summary>Click to reveal answer</summary>
LoRA (Low-Rank Adaptation) makes training much more efficient:
- Uses ~10x less memory
- Trains only ~0.1% of parameters
- Enables training 7B+ models on consumer GPUs
- Preserves most of the base model's knowledge
</details>

**4. What does the beta parameter control in DPO?**
<details>
<summary>Click to reveal answer</summary>
Beta (β) controls how strongly the model is penalized for deviating from the reference model:
- Higher β: Stronger constraint, stays closer to reference
- Lower β: Weaker constraint, more aggressive preference learning
- Default 0.1 is a good starting point
</details>

**5. When would you use PPOTrainer instead of DPOTrainer?**
<details>
<summary>Click to reveal answer</summary>
Use PPOTrainer when:
- You need online learning (continually generating and improving)
- You want more control over the reward function
- You have a pre-trained reward model you want to use
- You need complex reward shaping (combining multiple signals)

For most projects, DPOTrainer is simpler and sufficient!
</details>

---
## What's Next?

Now you understand TRL's toolkit! In the next notebook, we'll put it all together with a complete end-to-end RLHF pipeline.

**Continue to:** [Notebook 6: Full RLHF Pipeline](06_fine_tuning_with_rlhf.ipynb)

---

*TRL: "Don't reinvent the wheel - use the same tools that built ChatGPT!"*